### JsonOutputParser
* 사용자가 원하는 JSON 스키마를 지정할 수 있게 해주는 parsrer

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0, model_name="gpt-4o")

In [8]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

# 데이터 구조 정의
class Topic(BaseModel):
    description: str = Field(description="주제에 대한 간결한 설명")
    hashtags : str = Field(description="해시태그 형식의 키워드(2개 이상)")

# 파서 설정
output_parser = JsonOutputParser(pydantic_object=Topic)

In [9]:
print(output_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"description": {"description": "\uc8fc\uc81c\uc5d0 \ub300\ud55c \uac04\uacb0\ud55c \uc124\uba85", "title": "Description", "type": "string"}, "hashtags": {"description": "\ud574\uc2dc\ud0dc\uadf8 \ud615\uc2dd\uc758 \ud0a4\uc6cc\ub4dc(2\uac1c \uc774\uc0c1)", "title": "Hashtags", "type": "string"}}, "required": ["description", "hashtags"]}
```


In [10]:
from langchain_core.prompts import ChatPromptTemplate

question = "점심먹고 졸린 이유를 알려주세요."

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","당신은 친절한 AI 어시스턴트입니다. 질문에 간결하게 대답하세요."),
        ("user","Format:{format_instructions}\nQuestion:\n{question}")
    ]
)

prompt = prompt.partial(format_instructions = output_parser.get_format_instructions)


In [11]:
chain = prompt | model | output_parser

answer = chain.invoke({"question":question})

In [13]:
type(answer)

dict

In [12]:
print(answer)

{'description': '점심 식사 후 졸림은 주로 혈당 수치의 변화와 소화 과정으로 인한 에너지 소모 때문입니다. 식사 후 혈당이 급격히 상승하면 인슐린 분비가 증가하고, 이는 트립토판의 뇌 유입을 촉진하여 세로토닌과 멜라토닌 생성에 영향을 미쳐 졸음을 유발할 수 있습니다.', 'hashtags': '#점심졸림 #혈당변화'}


### Pydantic 을 사용하지 않고 JsonOutputParser 사용

In [17]:
output_parser = JsonOutputParser()

question = "잠에서 꺠는 방법에 대해 알려주세요. 방법에 대한 설명은 `description`에 관련키워드는 `hashtags`에 담아주세요."

#prompt
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","당신은 친절한 AI 어시스턴트입니다. 질문에 간결하게 대답하세요."),
        ("user","Format:{format_instructions}\nQuestion:\n{question}")
    ]
)

prompt = prompt.partial(format_instructions = output_parser.get_format_instructions())

chain= prompt | model | output_parser

response = chain.invoke({"question" : question})

print(response)

{'description': '잠에서 깨는 방법에는 여러 가지가 있습니다. 첫째, 알람을 설정하여 규칙적인 시간에 일어나는 습관을 들이는 것이 중요합니다. 둘째, 자연광을 활용하여 아침 햇살을 받으면 몸이 자연스럽게 깨어납니다. 셋째, 스트레칭이나 가벼운 운동을 통해 몸을 깨우는 것도 효과적입니다. 마지막으로, 물 한 잔을 마시면 몸이 수분을 보충하며 깨어날 수 있습니다.', 'hashtags': ['#알람', '#자연광', '#스트레칭', '#아침운동', '#수분보충']}
